In [204]:
import os
import numpy as np
import pandas
import time
#import random
import matplotlib
import matplotlib.pyplot as plt
#import scipy.stats
from sklearn.linear_model import LinearRegression

In [205]:
my_data = np.genfromtxt('qsar_fish_toxicity.csv', delimiter=';')

6 molecular descriptors and 1 quantitative experimental response:
1) CIC0
2) SM1_Dz(Z)
3) GATS1i
4) NdsCH
5) NdssC
6) MLOGP
7) quantitative response, LC50 [-LOG(mol/L)]

The linear regression model is given by LC50 = $\alpha_1$CIC0 + $\alpha_2$SM1_Dz(Z) + $\alpha_3$GATS1i + $\alpha_4$MLOGP + β

## 1) sklearn

In [206]:
X = my_data[:,[0,1,2,5]]
y = my_data[:,6]
reg = LinearRegression().fit(X, y)
print('alpha 1-4:')
print(reg.coef_)
print('beta:')
print(reg.intercept_)

alpha 1-4:
[ 0.44750162  1.22068139 -0.77463965  0.38310065]
beta:
2.1943526381758227


## 2) explicit formula
We have the model

\begin{equation}
\mathbf{y} = \mathbf{X}\mathbf{\alpha} + \mathbf{\epsilon}
\end{equation}

where
\begin{equation}
\mathbf{X} = \begin{bmatrix}
    1 & x_{11}       & x_{12} & x_{13} & x_{14} \\
    1 & x_{21}       & x_{22} & x_{23} & x_{24} \\
    ... & ... & ... & ... & ...\\
    1 & x_{n1}       & x_{n2} & x_{n3} & x_{n4}
\end{bmatrix}
\end{equation}

The first column corresponds to the constant term, and 2nd to 5th column correponds to CIC0, SM1_Dz(Z), GATS1i and MLOGP. The row corresponds to observations. 

\begin{equation}
\mathbf{\alpha} = \begin{bmatrix}
\beta \\
\alpha_1 \\
\alpha_2 \\
\alpha_3 \\
\alpha_4
\end{bmatrix}
\end{equation}

\begin{equation}
\mathbf{\epsilon} = \begin{bmatrix}
\epsilon_1 \\
\epsilon_2 \\
... \\
\epsilon_n
\end{bmatrix}
\end{equation}

The goal is to find $\tilde{\alpha}$ to minimize squared error $\sum_i^n \epsilon_i^2 = \epsilon'\epsilon  = (\mathbf{y} - \mathbf{X}\mathbf{\alpha})'(\mathbf{y} - \mathbf{X}\mathbf{\alpha})$

By matrix calculus, $\frac{ \partial {(\mathbf{y} - \mathbf{X}\mathbf{\alpha})'(\mathbf{y} - \mathbf{X}\mathbf{\alpha})}}{\partial{\mathbf{\alpha}} } = 2(\mathbf{y} - \mathbf{X}\mathbf{\alpha})'(-\mathbf{X})$

To find the minimum, we need to let $\frac{ \partial {(\mathbf{y} - \mathbf{X}\mathbf{\alpha})'(\mathbf{y} - \mathbf{X}\mathbf{\alpha})}}{\partial{\mathbf{\alpha}} } = 0$, and we have $(\mathbf{y} - \mathbf{X}\mathbf{\alpha})'\mathbf{X} = 0$

Transpose it, we have $\mathbf{X}'(\mathbf{y} - \mathbf{X}\mathbf{\alpha}) = 0$

To solve the optimal $\tilde{\alpha}$, we have $\mathbf{X}'\mathbf{y} = \mathbf{X}'\mathbf{X}\tilde{\alpha}$

Therefore, $\tilde{\alpha} = (\mathbf{X}'\mathbf{X})^{-1}\mathbf{X}'\mathbf{y}$


https://en.wikipedia.org/wiki/Matrix_calculus


In [207]:
X1 = np.ones(len(y))
X1 = np.reshape(X1, (len(y), 1))
X = np.array(X)
X_new = np.hstack((X1,X))

y = np.reshape(y, (len(y), 1))

In [208]:
alpha_left = np.linalg.inv(np.matmul(X_new.transpose(), X_new))
alpha_right = np.matmul(X_new.transpose(), y)
alpha = np.matmul(alpha_left, alpha_right)

In [209]:
print('alpha 1-4:')
print(alpha[1:])
print('beta')
print(alpha[0])

alpha 1-4:
[[ 0.44750162]
 [ 1.22068139]
 [-0.77463965]
 [ 0.38310065]]
beta
[2.19435264]


## 3) gradient descent method
Recall the notation:

\begin{equation}
\mathbf{\epsilon} = \begin{bmatrix}
\epsilon_1 \\
\epsilon_2 \\
... \\
\epsilon_n
\end{bmatrix}
\end{equation}

\begin{equation}
\mathbf{X} = \begin{bmatrix}
    1 & x_{11}       & x_{12} & x_{13} & x_{14} \\
    1 & x_{21}       & x_{22} & x_{23} & x_{24} \\
    ... & ... & ... & ... & ...\\
    1 & x_{n1}       & x_{n2} & x_{n3} & x_{n4}
\end{bmatrix}
\end{equation}

The algorithm for gradient descent is:
1) initialize with a coeffient matrix $\mathbf{\alpha_0}$

2) calculate the errors:
\begin{equation*}
\epsilon = \mathbf{y} - \mathbf{X}\mathbf{\alpha_i}
\end{equation*}

3) calculate the gradient:
\begin{equation*}
D\Lambda(\mathbf{\alpha_i}) = −\frac{2}{N} \epsilon'\mathbf{X}
\end{equation*}

4) update the coeffient matrix:
\begin{equation*}
\mathbf{\alpha_{i+1}} = \mathbf{\alpha_i} - D\Lambda(\mathbf{\alpha_i})
\end{equation*}

repeat (2)(3)(4) until the change in squared error $\epsilon'\epsilon$ is smaller than a predefined value (here we set it to $10^{-7}$)

### initializing with all 1 coefficient matrix

In [214]:
y = np.reshape(y, (len(y), 1))

delta = 0.02

#initialize the coefficient matrix
alpha_0 = np.ones(5)
alpha_0 = np.reshape(alpha_0, (5,1))

change = 1e10

while change > 1e-5:
    #calculate residue vector
    epsilon_0 = np.subtract(y, np.matmul(X_new, alpha_0))
    
    #calcualte gradient
    gradient = -2/len(y)*np.matmul(epsilon_0.transpose(), X_new)
    
    #update coefficient matrix
    alpha_1 = alpha_0 - delta*np.reshape(gradient, (5,1))
    
    #calculate the residue vector for new coefficient 
    epsilon_1 = np.subtract(y, np.matmul(X_new, alpha_1))
    
    #calculate change in squared error
    change = np.inner(epsilon_0.transpose(),epsilon_0.transpose()) - np.inner(epsilon_1.transpose(),epsilon_1.transpose())

    alpha_0 = alpha_1;
'''
    if (change>0):
        alpha_0 = alpha_1;
    else:
        print(alpha_1)
        alpha_0 = alpha_0; 
        break;
'''

'\n    if (change>0):\n        alpha_0 = alpha_1;\n    else:\n        print(alpha_1)\n        alpha_0 = alpha_0; \n        break;\n'

In [215]:
print('alpha 1-4:')
print(alpha_0[1:])
print('beta')
print(alpha_0[0])

alpha 1-4:
[[ 0.44903569]
 [ 1.22319641]
 [-0.7705289 ]
 [ 0.38344126]]
beta
[2.18194711]
